<a href="https://colab.research.google.com/github/williamweem/one-notebook-ds-project/blob/master/Ujian_Dicoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://drive.google.com/file/d/17QRz8aXbotrKXzuRBsFxhoXBNlMdm_mP/view?usp=sharing" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
import time
from google.colab import files
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

import tensorflow as tf

In [0]:
!wget --no-check-certificate  \
 https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
 -O /tmp/rockpaperscissors.zip


local_zip = "/tmp/rockpaperscissors.zip"
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [0]:
def to_label(value):
    if all(value==[1.,0.,0.]):
        return 'paper'
    elif all(value==[0.,1.,0.]):
        return "rock"
    else:
        return 'scissors'

def plots(ims, figsize=(24,12), rows=4, interp=False, titles=None):
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=20)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

def plot_accuracy(history,title):
    plt.title(title)
    plt.plot(history.history.history['accuracy'])
    plt.plot(history.history.history['val_accuracy'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()

def plot_loss(history,title):
    plt.title(title)
    plt.plot(history.history.history['loss'])
    plt.plot(history.history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()

In [0]:
batch_size = 20
nbr_epochs = 10
classnames = ['paper', 'rock', 'scissors']
root_dir = '/tmp/rockpaperscissors'

In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = "nearest",
    validation_split = 0.2
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = "nearest"
)

In [0]:
train_generator = train_datagen.flow_from_directory(
    root_dir,
    target_size = (200,200),
    batch_size = batch_size,
    class_mode = "categorical",
    shuffle = True,
    subset = "training",
    classes = classnames)

val_generator = train_datagen.flow_from_directory(
    root_dir,
    target_size = (200,200),
    batch_size = batch_size,
    class_mode = "categorical",
    shuffle=True,
    subset="validation",
    classes = classnames)

test_generator = test_datagen.flow_from_directory(
    root_dir,
    target_size = (200,200),
    batch_size = batch_size,
    class_mode = "categorical",
    shuffle=True,
    classes = classnames)

In [0]:
imgs, labels = next(train_generator)
imgs_label = list(map(to_label,labels))
plots(imgs, titles=imgs_label)

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(3, activation="softmax")
])

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [0]:
model.summary()

In [0]:
ts = time.time()

best_model_filename = 'weights.h5'   
best_model = ModelCheckpoint(best_model_filename, monitor='val_accuracy', verbose=1, save_best_only=True)

model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//batch_size,
    epochs = nbr_epochs,
    validation_data = val_generator,
    validation_steps = val_generator.samples//batch_size,
    callbacks = best_model
)

time.time() - ts

In [0]:
plot_accuracy(model,'CNN')
plot_loss(model,'CNN')

In [0]:
test_imgs, test_labels = next(test_generator)
predictions = model.predict(test_imgs)

df = pd.DataFrame()
df['actual'] = np.apply_along_axis(to_label, 1, test_labels)
df['predicted'] = np.apply_along_axis(to_label, 1, np.apply_along_axis(np.round, 1, predictions))
test_imgs = np.array(test_imgs).astype(np.uint8)
plots(test_imgs, titles=df['predicted'])

In [0]:
uploaded = files.upload()
 
for fn in uploaded.keys():
 
    # predicting images
    path = fn
    img = tf.keras.preprocessing.image.load_img(path, target_size=(200,200))
    imgplot = plt.imshow(img)
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    classes = model.predict(x, batch_size=10)
    print(np.apply_along_axis(to_label, 1, classes)[0])